Importing data

In [ ]:
import os
import torch
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from PIL import Image
import kagglehub


# Downloading dataset
als_data = kagglehub.dataset_download("uraninjo/augmented-alzheimer-mri-dataset")
dataset_path = als_data  # Path where dataset is stored
#print("Path to dataset files:", als_data)

Checking for corrupted files, using a function. Removes images if corrupted

In [ ]:
def is_corrupt(image_path):
    try:
        img = Image.open(image_path)
        img.verify()  # Check if image is readable
        return False
    except Exception:
        return True

# Remove corrupted images
image_paths = []
for root, _, files in os.walk(dataset_path):
    for file in files:
        if file.endswith(('jpg', 'png', 'jpeg')):
            full_path = os.path.join(root, file)
            if not is_corrupt(full_path):
                image_paths.append(full_path)
            else:
                os.remove(full_path)  # Delete corrupted images

Checks images for each class by counting how many images exist for each class. EX: (Early, Mild, Severe)

In [ ]:
dataset = datasets.ImageFolder(root=dataset_path)
class_counts = {class_name: 0 for class_name in dataset.classes} #dictionary to store class names
for _, label in dataset.samples:
    class_counts[dataset.classes[label]] += 1
print("Class Distribution:", class_counts)

Class Distribution: {'AugmentedAlzheimerDataset': 33984, 'OriginalDataset': 6400}


Define transformations for data augmentation & normalization


In [ ]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize images to 128x128 to ensure consistent size;
    #needed bc neural networks require fixed sized inputs, also helps with memory effciency
    transforms.RandomHorizontalFlip(),  # Random flip for data augmentation
    #helps prevent overfitting by increasing the training data, eliminates bias
    transforms.RandomRotation(5),  # Small rotation for variation(around 5 degrees)
    #makes small variations making the model more robust to slight positional changes
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # Adjust brightness & contrast (20%)
    #makes the model resilient to real-world conditions where lighting isn't always ideal
    transforms.ToTensor(),  # Convert image to PyTorch tensor, changes pixel values from  0-255 to a range of 0-1
    #PyTorch models only accept tensors, so this step is essential
    #Scaling pixel values from 0-255 to 0-1 helps improve numerical stability during training.
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize pixel values to [-1,1],
    #uses newpixel=orignialpixel-mean/std
    #Normalization helps stabilize training and speeds up convergence by ensuring that pixel values are centered around zero.
    #Many deep learning models (e.g., CNNs) work better when inputs are in a standardized range.

])

Load dataset using ImageFolder

In [ ]:
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

Split dataset into training (80%) and validation (20%)


In [ ]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

Create DataLoaders for batching and shuffling


In [ ]:
batch_size = 32  # Adjust if needed based on hardware
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

Baseline Model for CNN's

Print dataset info


In [ ]:
print(f"Total images: {len(dataset)}")
print(f"Training images: {len(train_dataset)}, Validation images: {len(val_dataset)}")
print(f"Classes: {dataset.classes}")  # Print class labels

Total images: 40384
Training images: 32307, Validation images: 8077
Classes: ['AugmentedAlzheimerDataset', 'OriginalDataset']


## Model Training: Convolutional Neural Networks


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import os
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Select device: Use GPU if available, otherwise fallback to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True  # Enable faster training on CUDA

# Define the CNN model class
class CNNBaseline(nn.Module):
    def __init__(self, num_classes):
        super(CNNBaseline, self).__init__()

        # First convolutional layer: 3 input channels (RGB), 32 filters, 3x3 kernel
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        # Second convolutional layer: 32 input channels, 64 filters, 3x3 kernel
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)

        # Max pooling layer with 2x2 kernel and stride of 2 (reduces spatial dimensions by half)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Compute flattened size after two pooling layers
        self.flattened_size = 64 * (64 // 4) * (64 // 4)  # 64 filters, image reduced to 16x16

        # Fully connected layers
        self.fc1 = nn.Linear(self.flattened_size, 128)  # First fully connected layer
        self.fc2 = nn.Linear(128, num_classes)          # Output layer

        # ReLU activation function
        self.relu = nn.ReLU()

    def forward(self, x):
        # Apply convolution, ReLU, and pooling for feature extraction
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))

        # Flatten the output for the fully connected layers
        x = x.view(x.size(0), -1)

        # Fully connected layers with activation
        x = self.relu(self.fc1(x))
        x = self.fc2(x)  # Final output (raw scores)
        return x

# Hyperparameters
batch_size = 64         # Number of samples per batch
epochs = 5              # Total number of training epochs
learning_rate = 0.001   # Learning rate for optimizer

# Path to dataset (assigned earlier in the notebook)
dataset_path = als_data

# Define data preprocessing and augmentation pipeline
transform = transforms.Compose([
    transforms.Resize((64, 64)),                     # Resize all images to 64x64
    transforms.RandomHorizontalFlip(),               # Randomly flip images horizontally
    transforms.RandomRotation(5),                    # Random rotation within 5 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # Brightness/contrast adjustments
    transforms.ToTensor(),                           # Convert image to PyTorch tensor
    transforms.Normalize(mean=[0.5], std=[0.5])      # Normalize to range [-1, 1]
])

# Load dataset from directory with transformations
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# Get class names and number of output classes
class_names = dataset.classes
n_classes = len(class_names)

# Use only 20% of dataset to speed up training during development
data_subset = int(0.2 * len(dataset))
train_dataset, _ = random_split(dataset, [data_subset, len(dataset) - data_subset])
test_dataset, _ = random_split(dataset, [int(0.2 * len(dataset)), len(dataset) - int(0.2 * len(dataset))])

# Create data loaders for training and testing
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize CNN model and move it to the selected device
model = CNNBaseline(num_classes=n_classes).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Adam optimizer

# Training loop
for epoch in range(epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to device
        optimizer.zero_grad()  # Reset gradients
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights
        running_loss += loss.item()  # Accumulate loss

    # Print average loss for the epoch
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

# Save trained model to disk
torch.save(model.state_dict(), 'cnn_baseline.pth')
print("Training complete. Model saved.")

# Evaluation mode
model.eval()
predictions, true_labels = [], []

# Disable gradient computation during evaluation
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)  # Forward pass
        _, predicted = torch.max(outputs, 1)  # Get predicted class with highest score
        predictions.extend(predicted.cpu().numpy())  # Store predictions
        true_labels.extend(labels.cpu().numpy())  # Store ground truth labels

# Calculate evaluation metrics
accuracy = accuracy_score(true_labels, predictions)  # Overall accuracy
conf_matrix = confusion_matrix(true_labels, predictions)  # Confusion matrix
class_report = classification_report(true_labels, predictions, target_names=class_names)  # Detailed report

# Print evaluation results
print(f"Test Accuracy: {accuracy:.4f}")
print("Confusion Matrix:")
#print(conf_matrix)
print("Classification Report:")
print(class_report)

# Optional: visualize the confusion matrix as a heatmap
#plt.figure(figsize=(8, 6))
#sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
#plt.xlabel('Predicted')
#plt.ylabel('Actual')
#plt.title('Confusion Matrix')
#plt.show()


Epoch 1, Loss: 0.3662
Epoch 2, Loss: 0.2826
Epoch 3, Loss: 0.2474
Epoch 4, Loss: 0.2202
Epoch 5, Loss: 0.2105
Training complete. Model saved.
Test Accuracy: 0.8618
Confusion Matrix:
Classification Report:
                           precision    recall  f1-score   support

AugmentedAlzheimerDataset       0.87      0.98      0.92      6766
          OriginalDataset       0.74      0.23      0.35      1310

                 accuracy                           0.86      8076
                macro avg       0.80      0.61      0.64      8076
             weighted avg       0.85      0.86      0.83      8076



# New CNN Model for all four class

Class Mapping

In [ ]:
print(dataset.classes)
# Output: ['Mild Demented', 'Moderate Demented', 'Non Demented', 'Very Mild Demented']


['AugmentedAlzheimerDataset', 'OriginalDataset']


Evaluation with Class Names

In [ ]:
# Class names from dataset
class_names = dataset.classes

# Compute accuracy and classification report
accuracy = accuracy_score(true_labels, predictions)
conf_matrix = confusion_matrix(true_labels, predictions)
class_report = classification_report(true_labels, predictions, target_names=class_names)

print(f"Test Accuracy: {accuracy:.4f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)


Test Accuracy: 0.8618
Confusion Matrix:
[[6657  109]
 [1007  303]]
Classification Report:
                           precision    recall  f1-score   support

AugmentedAlzheimerDataset       0.87      0.98      0.92      6766
          OriginalDataset       0.74      0.23      0.35      1310

                 accuracy                           0.86      8076
                macro avg       0.80      0.61      0.64      8076
             weighted avg       0.85      0.86      0.83      8076



## CNN Model For each Specific File

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# === STEP 1: Define paths to image folders for each category ===
base_dir = 'C:/Users/Facooney99/Downloads/ALSMRIScans.zip/OriginalDataset'
category_paths = {
    'Mild Demented': os.path.join(base_dir, 'MildDemented'),
    'Moderate Demented': os.path.join(base_dir, 'ModerateDemented'),
    'Non Demented': os.path.join(base_dir, 'NonDemented'),
    'Very Mild Demented': os.path.join(base_dir, 'VeryMildDemented')
}

# === STEP 2: Function to create image generators for each category ===
def create_data_generator(data_dir, target_size=(128, 128), batch_size=32):
    datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
    train_gen = datagen.flow_from_directory(data_dir, target_size=target_size,
                                            batch_size=batch_size, class_mode='categorical',
                                            subset='training')
    val_gen = datagen.flow_from_directory(data_dir, target_size=target_size,
                                          batch_size=batch_size, class_mode='categorical',
                                          subset='validation')
    return train_gen, val_gen

# === STEP 3: Function to build a simple CNN model ===
def build_cnn_model(input_shape=(128, 128, 3), num_classes=1):
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        MaxPooling2D(2,2),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# === STEP 4: Train a CNN for each category ===
models = {}
histories = {}

for category, path in category_paths.items():
    print(f"\n--- Training CNN for: {category} ---")
    train_gen, val_gen = create_data_generator(path)
    model = build_cnn_model(num_classes=train_gen.num_classes)
    history = model.fit(train_gen, validation_data=val_gen, epochs=10)

    models[category] = model
    histories[category] = history



--- Training CNN for: Mild Demented ---


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Facooney99/Downloads/ALSMRIScans.zip/OriginalDataset/MildDemented'